In [1]:
import numpy as np
import _pickle as cPickle
import gzip
import random

In [2]:
class Network(object):
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data = None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {}: {} / {}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} complete".format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        self.weights = [w - (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x]
        zs = []
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int (x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

In [3]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [4]:
def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding = 'latin1')
    f.close()
    
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [5]:
training_data, validation_data, test_data = load_data_wrapper()

In [6]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data = test_data)

Epoch 0: 8203 / 10000
Epoch 1: 8368 / 10000
Epoch 2: 8402 / 10000
Epoch 3: 8487 / 10000
Epoch 4: 8489 / 10000
Epoch 5: 8497 / 10000
Epoch 6: 9362 / 10000
Epoch 7: 9357 / 10000
Epoch 8: 9388 / 10000
Epoch 9: 9412 / 10000
Epoch 10: 9426 / 10000
Epoch 11: 9422 / 10000
Epoch 12: 9465 / 10000
Epoch 13: 9465 / 10000
Epoch 14: 9409 / 10000
Epoch 15: 9450 / 10000
Epoch 16: 9458 / 10000
Epoch 17: 9465 / 10000
Epoch 18: 9466 / 10000
Epoch 19: 9472 / 10000
Epoch 20: 9465 / 10000
Epoch 21: 9485 / 10000
Epoch 22: 9459 / 10000
Epoch 23: 9458 / 10000
Epoch 24: 9479 / 10000
Epoch 25: 9465 / 10000
Epoch 26: 9493 / 10000
Epoch 27: 9462 / 10000
Epoch 28: 9490 / 10000
Epoch 29: 9484 / 10000


In [7]:
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data = test_data)

Epoch 0: 6566 / 10000
Epoch 1: 6691 / 10000
Epoch 2: 6764 / 10000
Epoch 3: 6757 / 10000
Epoch 4: 6749 / 10000
Epoch 5: 6738 / 10000
Epoch 6: 6817 / 10000
Epoch 7: 6856 / 10000
Epoch 8: 6839 / 10000
Epoch 9: 6858 / 10000
Epoch 10: 6888 / 10000
Epoch 11: 6862 / 10000
Epoch 12: 6896 / 10000
Epoch 13: 6894 / 10000
Epoch 14: 6907 / 10000
Epoch 15: 6907 / 10000
Epoch 16: 6935 / 10000
Epoch 17: 6931 / 10000
Epoch 18: 6946 / 10000
Epoch 19: 6954 / 10000
Epoch 20: 6980 / 10000
Epoch 21: 6968 / 10000
Epoch 22: 7000 / 10000
Epoch 23: 6986 / 10000
Epoch 24: 7036 / 10000
Epoch 25: 7036 / 10000
Epoch 26: 7203 / 10000
Epoch 27: 7826 / 10000
Epoch 28: 7847 / 10000
Epoch 29: 7885 / 10000


In [8]:
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 0.001, test_data = test_data)

Epoch 0: 1053 / 10000
Epoch 1: 1241 / 10000
Epoch 2: 1366 / 10000
Epoch 3: 1424 / 10000
Epoch 4: 1470 / 10000
Epoch 5: 1494 / 10000
Epoch 6: 1522 / 10000
Epoch 7: 1546 / 10000
Epoch 8: 1564 / 10000
Epoch 9: 1582 / 10000
Epoch 10: 1596 / 10000
Epoch 11: 1618 / 10000
Epoch 12: 1640 / 10000
Epoch 13: 1657 / 10000
Epoch 14: 1676 / 10000
Epoch 15: 1700 / 10000
Epoch 16: 1716 / 10000
Epoch 17: 1735 / 10000
Epoch 18: 1761 / 10000
Epoch 19: 1788 / 10000
Epoch 20: 1805 / 10000
Epoch 21: 1823 / 10000
Epoch 22: 1854 / 10000
Epoch 23: 1871 / 10000
Epoch 24: 1889 / 10000
Epoch 25: 1913 / 10000
Epoch 26: 1932 / 10000
Epoch 27: 1964 / 10000
Epoch 28: 1996 / 10000
Epoch 29: 2031 / 10000
